In [13]:
import cobra.test
import pandas as pd
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit
import numpy as np
import seaborn as sns
import os
import math
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model


### Determine Maintenance energy for each biolgoically relvenat combination of the electron transport system

To do this we will have a set of enzyme combinations to make a uqiue path that will share electrons between O2 reduction and N2 reduction. Each path will have a diffrenent stoichiometery for proton tanslocation leading to differnet sucrose per ATP ratio. So given a the experimentally determined maintenance coefficent (mmol of scurose/ hr/ gCDW) different maintence rates (ATPM)  will be given for each ETS pathway. The assumption that will be made is that a minimizing of the ATP maintence is perfered (more on this assumption later). So each ETS path will have a coressponding ATPM flux in each O2 concentration we can then use this ATPM flux (determined from experimental maintence coefficent) and make it a constant in a simulation with experimentally determined sucrose uptake rate. With the given sucrose uptake rate and the determined ATPM flux we can test how similar the predicted growth is to the experimental growth. 

By looking at the branched diagram below we can divde the ETS paths into 8 paths each with the NADHI/NADHII, RNF/FIX, or Cytbd/cytco pairs. 



<img src="Images/branched.PNG" width=800 height=400 />

For nomeclature we will simplfy names as follows: 

- **NADH5**- NADH dehydrogenase II (uncoupled) = NII
- **NADH6**- NADH dehyrdogenase I (coupled) = NI
- **FIX**- Fix = F
- **RNF**- Rnf = R
- **CYTBDpp**- Cytochrome bd = BD
- **CYOO2pp**- Cytochrome c pathways (complex III and IV) = CO

So there are 8 combinations 

- NII_BD_F
- NII_CO_F
- NII_BD_R
- NII_CO_R
- NI_BD_F
- NI_CO_F
- NI_BD_R
- NI_CO_R

*We know that there will be overlap in total H+/e- translocated and that some of these combinations are not biologically relvent (NII CO) but looking at all combinations allow for interpretations of "inbetween" states*

Now to add some confusion to nomenclature we will have to knockout the enzymes not in the path. So we will make a list of reacion ids of enzymes not in the path. 

In [86]:
#reaction ids not in pathway then "name" of pathway
NII_BD_F = ["NDHI", "CYOO2pp", "RNF", "NII_BD_F"]
NII_CO_F = ["NDHI", "CYTBDpp", "RNF", "NII_CO_F"]
NII_BD_R = ["NDHI", "CYOO2pp", "FIX", "NII_BD_R"]
NII_CO_R = ["NDHI", "CYTBDpp", "FIX", "NII_CO_R"]
NI_BD_F = ["NADH5", "CYTBDpp", "RNF", "NI_BD_F"]
NI_CO_F = ["NADH5", "CYTBDpp", "RNF", "NI_CO_F"] 
NI_BD_R = ["NADH5", "CYOO2pp", "FIX", "NI_BD_R"]
NI_CO_R= ["NADH5", "CYTBDpp", "FIX", "NI_CO_R"]

all_paths = [NII_BD_F,
                NII_CO_F, 
                NII_BD_R,
                NII_CO_R,
                NI_BD_F,
                NI_CO_F,
                NI_BD_R,
                NI_CO_R]

In [87]:
all_paths

[['NDHI', 'CYOO2pp', 'RNF', 'NII_BD_F'],
 ['NDHI', 'CYTBDpp', 'RNF', 'NII_CO_F'],
 ['NDHI', 'CYOO2pp', 'FIX', 'NII_BD_R'],
 ['NDHI', 'CYTBDpp', 'FIX', 'NII_CO_R'],
 ['NADH5', 'CYTBDpp', 'RNF', 'NI_BD_F'],
 ['NADH5', 'CYTBDpp', 'RNF', 'NI_CO_F'],
 ['NADH5', 'CYOO2pp', 'FIX', 'NI_BD_R'],
 ['NADH5', 'CYTBDpp', 'FIX', 'NI_CO_R']]

Now we want to write a function that can determine the ATPM flux over under multiple diffrent O2 concentrations for each pathway. Frist lay out the maintenance coefficent from Khula and Oelze 1988 (in data folder):

In [88]:
#Add the maintenance coefficent values from Khula et al converted into mmol of sucrose / hr / gCDW 
Maintenance_coefficient = [0.9, 4.4, 6.2, 7, 8]

#Add the coresponding O2 concentration for labeling
O2_concentration = [12, 48, 108, 144, 192]

In [89]:
def Determine_maintenance(Model, rxn_path, lb, ub, interval, maint_coef, O2_conc, path_name ):
    
    '''
    This takes a cobra model in json format and knockouts reactions (sets bounds to zero) of intrest listed in rxn_path.
    For this version there is only three reactions to be changed. 
    
    The function interates over diffrent experimentally determined maintenance coefficents and predicts a ATP maintenance
    rates by creating a array of testable lower bounds for ATPM. When the growth rate is zero the ATPM flux matches the given
    maintenance coefficent in (Maint_coef) resulting in the ATPM for that condition (O2_conc). 
    
    Variables lb, ub, and interval are to be played with. If expected to have large maintenance that set ub to a large number
    (never exceed 1000). The high the interval the more accurate the ATPM rate when zero but also increases the iterations of 
    the function. 
    
    
    '''
    print(path_name)
    #create array of testable bounds
    ATPM_bound = np.linspace(lb, ub, interval).tolist()
    ATPM_bound.reverse()
    #print(ATPM_bound)
    ATPM_bound_value = []
    Growth_rate =[]
    Main_co = [] 
    
    
    for (j) in (maint_coef): 
        model = cobra.io.load_json_model(Model)
        
        #Make sure glucose is set to zero
        model.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
        model.reactions.get_by_id("EX_glc__D_e").upper_bound = 0
       
        #Make sure model is diazotrophic
        model.reactions.get_by_id("EX_nh4_e").lower_bound = 0
        model.reactions.get_by_id("EX_nh4_e").upper_bound = 0
        
        #set the flux to zero of corresponding reactions in pathway of interest
        model.reactions.get_by_id(rxn_path[0]).upper_bound = 0
        model.reactions.get_by_id(rxn_path[0]).lower_bound = 0

        model.reactions.get_by_id(rxn_path[1]).upper_bound = 0
        model.reactions.get_by_id(rxn_path[1]).lower_bound = 0
        
        model.reactions.get_by_id(rxn_path[2]).upper_bound = 0
        model.reactions.get_by_id(rxn_path[2]).lower_bound = 0
        
        #Set sucrose uptake to maintenance coefficent
        model.reactions.get_by_id("EX_sucr_e").lower_bound = -j
        model.reactions.get_by_id("EX_sucr_e").upper_bound = -j
        
        #save model if needed for extra analysis
        
        save_json_model(model, './Models/Maintenance_Models/MaintenanceRate_sucrose{}_O2_{}.json'.format(j, path_name))
        
        #This takes some time so nice to know what step its on 
        print(f'Running maintenance coefficient {j}')
         
        for i in ATPM_bound:
            model.reactions.get_by_id("ATPM").upper_bound = 1000
            model.reactions.get_by_id("ATPM").lower_bound = i
            solution = model.optimize()
            ATPM_bound_value.append(i)
            Growth_rate.append(solution.objective_value)
            Main_co.append(j)
            #O2_conc.append(k) ### removed adding oxygen concentraition here as it messed with array length
            
    #these were to make sure the arrays were of equal length
    #print(len(ATPM_bound_value))
    #print(len(Growth_rate))    
    #print(len(Main_co))
    #print(len(O2_conc)) 
    
    #Make data frame
    df = pd.DataFrame({
            #'O2_concentration': O2_conc,
            'Maintenance_coefficient':Main_co,
            'ATPM_bound_value':ATPM_bound_value,
            'Growth_Rate':Growth_rate})
    
    #add colummn with pathway label
    df.insert(0, 'ETS_path', path_name)
    
    #label row with oxygen concentration (I am sure there is a better way to do this)
    def o2_label(l):
        if l['Maintenance_coefficient'] == 0.9:
            return 12
        elif l['Maintenance_coefficient'] == 4.4:
            return 48
        elif l['Maintenance_coefficient'] == 6.2:
            return 108
        elif l['Maintenance_coefficient'] == 7.0:
            return 144
        else:
            return 192
    df['O2_concentration'] =df.apply(o2_label, axis=1)
    
    #save to csv with details on which path an which conditions were used 
    
    df.to_csv('MaintenanceRate_sucrose_O2_{}.csv'.format(path_name), sep=',', index=False)
    
    #output maintenance rate by determining where each condition hits zero
    print('Maintenance rates for pathway {} :'.format(path_name))
    df_12_limit = df[(df.Growth_Rate == 0) & (df.O2_concentration == 12)]
    print(f' ATP Maintenance at 12 \u03BC Mol of O2 is:{df_12_limit["ATPM_bound_value"].iloc[-1]}')
    df_48_limit = df[(df.Growth_Rate == 0) & (df.O2_concentration == 48)]
    print(f' ATP Maintenance at 48 \u03BC Mol of O2 is:{df_48_limit["ATPM_bound_value"].iloc[-1]}')
    df_108_limit = df[(df.Growth_Rate == 0) & (df.O2_concentration == 108)]
    print(f' ATP Maintenance at 108 \u03BC Mol of O2 is:{df_108_limit["ATPM_bound_value"].iloc[-1]}')
    df_144_limit = df[(df.Growth_Rate == 0) & (df.O2_concentration == 144)]
    print(f' ATP Maintenance at 144 \u03BC Mol of O2 is:{df_144_limit["ATPM_bound_value"].iloc[-1]}')
    df_192_limit = df[(df.Growth_Rate == 0) & (df.O2_concentration == 192)]
    print(f' ATP Maintenance at 192 \u03BC Mol of O2 is: {df_192_limit["ATPM_bound_value"].iloc[-1]}')
    ;
    

In [92]:
#check that it works for a single pathway
Determine_maintenance("./Models/model_n2_MFA.json", NII_BD_F, 0, 750, 100, Maintenance_coefficient, O2_concentration,  "NII_BD_F")

NII_BD_F
Running maintenance coefficient 0.9
Running maintenance coefficient 4.4
Running maintenance coefficient 6.2
Running maintenance coefficient 7
Running maintenance coefficient 8
Maintenance rates for pathway NII_BD_F :
 ATP Maintenance at 12 μ Mol of O2 is:22.727272727272727
 ATP Maintenance at 48 μ Mol of O2 is:83.33333333333334
 ATP Maintenance at 108 μ Mol of O2 is:113.63636363636364
 ATP Maintenance at 144 μ Mol of O2 is:128.78787878787878
 ATP Maintenance at 192 μ Mol of O2 is: 143.93939393939394


In [93]:
#iterate of all pathways 

for ETS_path in all_paths:
    Determine_maintenance("./Models/model_n2_MFA.json", ETS_path, 0, 750, 2000, Maintenance_coefficient, O2_concentration, ETS_path[3])

NII_BD_F
Running maintenance coefficient 0.9
Running maintenance coefficient 4.4
Running maintenance coefficient 6.2
Running maintenance coefficient 7
Running maintenance coefficient 8
Maintenance rates for pathway NII_BD_F :
 ATP Maintenance at 12 μ Mol of O2 is:16.133066533266632
 ATP Maintenance at 48 μ Mol of O2 is:78.41420710355177
 ATP Maintenance at 108 μ Mol of O2 is:110.68034017008503
 ATP Maintenance at 144 μ Mol of O2 is:124.93746873436717
 ATP Maintenance at 192 μ Mol of O2 is: 142.5712856428214
NII_CO_F
Running maintenance coefficient 0.9
Running maintenance coefficient 4.4
Running maintenance coefficient 6.2
Running maintenance coefficient 7
Running maintenance coefficient 8
Maintenance rates for pathway NII_CO_F :
 ATP Maintenance at 12 μ Mol of O2 is:37.89394697348674
 ATP Maintenance at 48 μ Mol of O2 is:184.21710855427713
 ATP Maintenance at 108 μ Mol of O2 is:259.2546273136568
 ATP Maintenance at 144 μ Mol of O2 is:293.0215107553777
 ATP Maintenance at 192 μ Mol of O

In [78]:
model = cobra.io.load_json_model("./Models/iAA1289.json")  
    
rxn_path = ["NADH6", "CYOO2pp", "RNF", "NII_BD_F"]

    
model.reactions.get_by_id(rxn_path[0]).upper_bound = 0
model.reactions.get_by_id(rxn_path[0]).lower_bound = 0

model.reactions.get_by_id(rxn_path[1]).upper_bound = 0
model.reactions.get_by_id(rxn_path[1]).lower_bound = 0
        
model.reactions.get_by_id(rxn_path[2]).upper_bound = 0
model.reactions.get_by_id(rxn_path[2]).lower_bound = 0

solution = model.optimize()
model.summary()

In [79]:
model.metabolites.o2_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,O2tpp,100.000000,14.617260,o2_p <=> o2_c
CONSUMING,CYTBDpp,99.998877,14.617096,2.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 2.0 h_...
